In [1]:
# General imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#plt.style.use('seaborn-v0_8')
import torch
import os

# MSE function 
def MSE(pred, true):
    return np.mean((pred-true)**2)

random_seed = 100
torch.manual_seed = 100
np.random.seed = 100


In [2]:
from InformerAPI import InformerTS
from CrossformerAPI import CrossformerTS
from AutoformerAPI import AutoformerTS
from FedformerAPI import FedformerTS
from PyraformerAPI import PyraformerTS
from LogSparseAPI import LogsparseTS
#from TSMixerAPI import TSMixer

### Looping models in one go

In [10]:
datasets = {
    'ETTh1': ['./ETTDataset/' , 'M' , 'OT' , 96 , 7 , 7 , 7] ,
    'DEWINDh_small': ['./WINDataset/' , 'S' , 'TARGET' , 168, 1 , 1 , 1] ,
    'SYNTHh1': ['./SYNTHDataset/', 'S' , 'TARGET' , 168, 1 , 1 , 1] ,
    'SYNTH_additive' : ['./SYNTHDataset/', 'S' , 'TARGET' , 168, 1 , 1 , 1] ,
    'SYNTH_additive_reversal' : ['./SYNTHDataset/', 'S' , 'TARGET' , 168, 1 , 1 , 1] ,
    'SYNTH_multiplicative' : ['./SYNTHDataset/', 'S' , 'TARGET' , 168, 1 , 1 , 1] ,
    'SYNTH_multiplicative_reversal' : ['./SYNTHDataset/', 'S' , 'TARGET' , 168, 1 , 1 , 1]
}
pred_lens = [24, 48, 168, 336, 720]
models = [AutoformerTS]


In [11]:
%%time

# Simple loop
for transformermodel in models:
    print(f'Model in use: {transformermodel.__name__}')
    model = transformermodel()
    model.compile(learning_rate=1e-4, loss='mse', early_stopping_patience=3)
    for iter in range(1,model.args.itr):
        for dataset_name, dataset_params in datasets.items():
            for pred_len in pred_lens:
                print(f"Training {transformermodel.__name__} on {dataset_name} with pred_len {pred_len}")

                    # High amount of epochs to accomodate all models
                    # Early stopping should kick in anyways
                model.fit(
                    iter = iter + 1,
                    data=dataset_name,
                    data_root_path=dataset_params[0],
                    batch_size=32,
                    epochs=20, # very high
                    pred_len=pred_len,
                    features = dataset_params[1],
                    target = dataset_params[2],
                    seq_len = dataset_params[3],
                    enc_in = dataset_params[4],
                    dec_in = dataset_params[5],
                    c_out = dataset_params[6]
                )

                predictions = model.predict()

                print(f'\nMoving to next...\n')


Model in use: AutoformerTS
Training AutoformerTS on ETTh1 with pred_len 24
Beginning to fit the model with the following arguments:
{'model': 'Autoformer', 'target': 'OT', 'des': 'Exp', 'dropout': 0.05, 'num_workers': 0, 'freq': 'h', 'checkpoints': './checkpoints/', 'bucket_size': 4, 'n_hashes': 4, 'is_trainging': True, 'root_path': './ETTDataset/', 'data_path': 'ETTh1.csv', 'data': 'ETTh1', 'features': 'M', 'seq_len': 96, 'label_len': 48, 'pred_len': 24, 'e_layers': 2, 'd_layers': 1, 'n_heads': 8, 'factor': 1, 'enc_in': 7, 'dec_in': 7, 'c_out': 7, 'd_model': 512, 'itr': 3, 'd_ff': 2048, 'moving_avg': 25, 'distil': True, 'output_attention': False, 'patience': 3, 'learning_rate': 0.0001, 'batch_size': 32, 'embed': 'timeF', 'activation': 'gelu', 'use_amp': False, 'loss': 'mse', 'train_epochs': 20, 'lradj': 'type1', 'model_id': 'Autoformer_Synth1_24', 'gpu': 0, 'devices': '0,1,2,3', 'use_multi_gpu': False, 'use_gpu': True, 'iter': 2}
Use GPU: cuda:0
>>>>>>>start training : Autoformer_ETTh

In [12]:

datasets = {
    'ETTh1': ['./ETTDataset/' , 'M' , 'OT' , 96 , 7 , 7 , 7] ,
    'DEWINDh_small': ['./WINDataset/' , 'S' , 'TARGET' , 96, 1 , 1 , 1] ,
    'SYNTHh1': ['./SYNTHDataset/', 'S' , 'TARGET' , 96, 1 , 1 , 1] ,
    'SYNTH_additive' : ['./SYNTHDataset/', 'S' , 'TARGET' , 96, 1 , 1 , 1] ,
    'SYNTH_additive_reversal' : ['./SYNTHDataset/', 'S' , 'TARGET' , 96, 1 , 1 , 1] ,
    'SYNTH_multiplicative' : ['./SYNTHDataset/', 'S' , 'TARGET' , 96, 1 , 1 , 1] ,
    'SYNTH_multiplicative_reversal' : ['./SYNTHDataset/', 'S' , 'TARGET' , 96, 1 , 1 , 1]
}
pred_lens = [24, 48, 168, 336, 720]

models = [FedformerTS]


In [13]:
%%time

# Simple loop
for transformermodel in models:
    print(f'Model in use: {transformermodel.__name__}')
    model = transformermodel()
    model.compile(learning_rate=1e-4, loss='mse', early_stopping_patience=3)
    for iter in range(model.args.itr):
        for dataset_name, dataset_params in datasets.items():
            for pred_len in pred_lens:
                print(f"Training {transformermodel.__name__} on {dataset_name} with pred_len {pred_len}")

                    # High amount of epochs to accomodate all models
                    # Early stopping should kick in anyways
                model.fit(
                    iter = iter + 1,
                    data=dataset_name,
                    data_root_path=dataset_params[0],
                    batch_size=32,
                    epochs=20, # very high
                    pred_len=pred_len,
                    features = dataset_params[1],
                    target = dataset_params[2],
                    seq_len = dataset_params[3],
                    enc_in = dataset_params[4],
                    dec_in = dataset_params[5],
                    c_out = dataset_params[6]
                )

                predictions = model.predict()

                print(f'\nMoving to next...\n')


Model in use: FedformerTS
Training FedformerTS on ETTh1 with pred_len 24
Beginning to fit the model with the following arguments:
{'model': 'Fedformer', 'version': 'Fourier', 'mode_select': 'random', 'modes': 64, 'L': 3, 'base': 'legendre', 'cross_activation': 'tanh', 'data': 'ETTh1', 'root_path': './ETTDataset/', 'data_path': 'ETTh1.csv', 'target': 'OT', 'checkpoints': './checkpoints/', 'features': 'M', 'freq': 'h', 'detail_freq': 'h', 'seq_len': 96, 'label_len': 48, 'pred_len': 24, 'enc_in': 7, 'dec_in': 7, 'c_out': 7, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'moving_avg': 25, 'factor': 1, 'distil': True, 'dropout': 0.05, 'embed': 'timeF', 'activation': 'gelu', 'output_attention': False, 'model_id': 'Fedformer_Synth1_24', 'num_workers': 0, 'itr': 3, 'train_epochs': 20, 'batch_size': 32, 'patience': 3, 'learning_rate': 0.0001, 'des': 'test', 'loss': 'mse', 'lradj': 'type1', 'use_amp': False, 'gpu': 0, 'devices': '0,1,2,3', 'use_multi_gpu': False, 'use_

In [8]:
datasets = {    
'SYNTH_multiplicative_reversal' : ['./SYNTHDataset/', 'S' , 'TARGET' , 768, 1 , 1 , 1]
}
pred_lens = [720]

In [20]:
%%time

# Simple loop
for transformermodel in models:
    print(f'Model in use: {transformermodel.__name__}')
    model = transformermodel()
    model.compile(learning_rate=1e-4, loss='mse', early_stopping_patience=3)
    for iter in range(1 ,2):
        for dataset_name, dataset_params in datasets.items():
            for pred_len in pred_lens:
                print(f"Training {transformermodel.__name__} on {dataset_name} with pred_len {pred_len}")

                        # High amount of epochs to accomodate all models
                        # Early stopping should kick in anyways
                model.fit(
                    iter = iter + 1,
                    data=dataset_name,
                    data_root_path=dataset_params[0],
                    batch_size=32,
                    epochs=20, # very high
                    pred_len=pred_len,
                    features = dataset_params[1],
                    target = dataset_params[2],
                    seq_len = dataset_params[3],
                    enc_in = dataset_params[4],
                    dec_in = dataset_params[5],
                    c_out = dataset_params[6]
                )

                predictions = model.predict()

                print(f'\nMoving to next...\n')


Model in use: LogsparseTS
Training LogsparseTS on SYNTH_multiplicative_reversal with pred_len 720
Beginning to fit the model with the following arguments:
{'model': 'Logsparse', 'plot_flat': 0, 'verbose': 1, 'is_training': 1, 'inverse': False, 'data': 'SYNTH_multiplicative_reversal', 'root_path': './SYNTHDataset/', 'data_path': 'SYNTH_multiplicative_reversal.csv', 'target': 'TARGET', 'freq': 'h', 'checkpoints': './checkpoints/', 'checkpoint_flag': 1, 'features': 'S', 'seq_len': 768, 'label_len': 48, 'pred_len': 720, 'enc_in': 1, 'dec_in': 1, 'c_out': 1, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'factor': 3, 'distil': True, 'dropout': 0.05, 'embed': 'timeF', 'activation': 'gelu', 'output_attention': False, 'win_len': 6, 'res_len': None, 'qk_ker': 4, 'v_conv': 0, 'sparse_flag': 1, 'top_keys': 0, 'kernel_size': 3, 'train_strat_lstm': 'recursive', 'model_id': 'Logsparse_Synth1_24', 'num_workers': 0, 'itr': 3, 'train_epochs': 20, 'batch_size': 32, 'patience':

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacty of 44.35 GiB of which 5.62 MiB is free. Process 384 has 1.29 GiB memory in use. Process 19020 has 9.40 GiB memory in use. Process 5885 has 33.63 GiB memory in use. Of the allocated memory 9.00 GiB is allocated by PyTorch, and 57.86 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [5]:
datasets = {
    'ETTh1': ['./ETTDataset/' , 'M' , 'OT' , 768 , 7 , 7 , 7] ,
    'DEWINDh_small': ['./WINDataset/' , 'S' , 'TARGET' , 768, 1 , 1 , 1] ,
    'SYNTHh1': ['./SYNTHDataset/', 'S' , 'TARGET' , 768, 1 , 1 , 1] ,
    'SYNTH_additive' : ['./SYNTHDataset/', 'S' , 'TARGET' , 768, 1 , 1 , 1] ,
    'SYNTH_additive_reversal' : ['./SYNTHDataset/', 'S' , 'TARGET' ,768, 1 , 1 , 1] ,
    'SYNTH_multiplicative' : ['./SYNTHDataset/', 'S' , 'TARGET' , 768, 1 , 1 , 1] ,
    'SYNTH_multiplicative_reversal' : ['./SYNTHDataset/', 'S' , 'TARGET' , 768, 1 , 1 , 1]
}
pred_lens = [24, 48, 168, 336, 720]

models = [LogsparseTS]


In [6]:
%%time

# Simple loop
for transformermodel in models:
    print(f'Model in use: {transformermodel.__name__}')
    model = transformermodel()
    model.compile(learning_rate=1e-4, loss='mse', early_stopping_patience=3)
    for iter in range(1 ,model.args.itr):
        for dataset_name, dataset_params in datasets.items():
            for pred_len in pred_lens:
                print(f"Training {transformermodel.__name__} on {dataset_name} with pred_len {pred_len}")

                        # High amount of epochs to accomodate all models
                        # Early stopping should kick in anyways
                model.fit(
                    iter = iter + 1,
                    data=dataset_name,
                    data_root_path=dataset_params[0],
                    batch_size=32,
                    epochs=20, # very high
                    pred_len=pred_len,
                    features = dataset_params[1],
                    target = dataset_params[2],
                    seq_len = dataset_params[3],
                    enc_in = dataset_params[4],
                    dec_in = dataset_params[5],
                    c_out = dataset_params[6]
                )

                predictions = model.predict()

                print(f'\nMoving to next...\n')


Model in use: LogsparseTS
Training LogsparseTS on ETTh1 with pred_len 24
Beginning to fit the model with the following arguments:
{'model': 'Logsparse', 'plot_flat': 0, 'verbose': 1, 'is_training': 1, 'inverse': False, 'data': 'ETTh1', 'root_path': './ETTDataset/', 'data_path': 'ETTh1.csv', 'target': 'OT', 'freq': 'h', 'checkpoints': './checkpoints/', 'checkpoint_flag': 1, 'features': 'M', 'seq_len': 768, 'label_len': 48, 'pred_len': 24, 'enc_in': 7, 'dec_in': 7, 'c_out': 7, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'factor': 3, 'distil': True, 'dropout': 0.05, 'embed': 'timeF', 'activation': 'gelu', 'output_attention': False, 'win_len': 6, 'res_len': None, 'qk_ker': 4, 'v_conv': 0, 'sparse_flag': 1, 'top_keys': 0, 'kernel_size': 3, 'train_strat_lstm': 'recursive', 'model_id': 'Logsparse_Synth1_24', 'num_workers': 0, 'itr': 3, 'train_epochs': 20, 'batch_size': 32, 'patience': 3, 'learning_rate': 0.0001, 'lr_decay_rate': 0.8, 'des': 'test', 'loss': 'mse',

OutOfMemoryError: CUDA out of memory. Tried to allocate 576.00 MiB. GPU 0 has a total capacty of 44.35 GiB of which 354.31 MiB is free. Process 384 has 1.29 GiB memory in use. Process 10346 has 1.28 GiB memory in use. Process 19020 has 7.78 GiB memory in use. Process 5885 has 33.63 GiB memory in use. Of the allocated memory 7.24 GiB is allocated by PyTorch, and 198.59 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
datasets = {
    'ETTh1': ['./ETTDataset/' , 'M'] ,
    'DEWINDh_small': ['./WINDataset/' , 'S'] ,
    'SYNTHh1': ['./SYNTHDataset/', 'S'] ,
    'SYNTH_additive' : ['./SYNTHDataset/', 'S'] ,
    'SYNTH_additive_reversal' : ['./SYNTHDataset/', 'S'] ,
    'SYNTH_multiplicative' : ['./SYNTHDataset/', 'S'] ,
    'SYNTH_multiplicative_reversal' : ['./SYNTHDataset/' , 'S']
}
pred_lens = [24, 48, 168, 336, 720]

models = [InformerTS]

In [11]:
%%time 

# Simple loop
for transformermodel in models:
    print(f'Model in use: {transformermodel.__name__}')
    model = transformermodel()
    model.compile(learning_rate=1e-4, loss='mse', early_stopping_patience=3)
    for iter in range(model.args.itr):
        for dataset_name, dataset_params in datasets.items():
            for pred_len in pred_lens:
                print(f"Training {transformermodel.__name__} on {dataset_name} with pred_len {pred_len}")

                # High amount of epochs to accomodate all models
                # Early stopping should kick in anyways
                model.fit(
                    iter = iter + 1 ,
                    data=dataset_name,
                    data_root_path=dataset_params[0],
                    batch_size=32,
                    epochs=20, # very high
                    pred_len=pred_len,
                    features = dataset_params[1],
                )

                predictions = model.predict()

                print(f'\nMoving to next...\n')


Model in use: InformerTS
Training InformerTS on ETTh1 with pred_len 24
Beginning to fit the model with the following arguments:
{'model': 'informerstack', 'features': 'M', 'target': 'OT', 'freq': 'h', 'checkpoints': './checkpoints', 'seq_len': 168, 'label_len': 48, 'enc_in': 7, 'dec_in': 7, 'c_out': 7, 'factor': 5, 'd_model': 512, 'n_heads': 8, 's_layers': [3, 2, 1], 'd_layers': 2, 'd_ff': 2048, 'dropout': 0.05, 'attn': 'prob', 'embed': 'timeF', 'activation': 'gelu', 'distil': True, 'output_attention': False, 'mix': True, 'padding': 0, 'lradj': 'type1', 'use_amp': False, 'num_workers': 0, 'itr': 3, 'des': 'exp', 'use_gpu': True, 'gpu': 0, 'use_multi_gpu': False, 'devices': '0,1,2,3', 'learning_rate': 0.0001, 'loss': 'mse', 'patience': 3, 'data': 'ETTh1', 'root_path': './ETTDataset/', 'data_path': 'ETTh1.csv', 'train_epochs': 20, 'batch_size': 32, 'pred_len': 24, 'iter': 2, 'detail_freq': 'h'}
Use GPU: cuda:0
>>>>>>>start training : informerstack_ETTh1_ftM_sl168_ll48_pl24_dm512_nh8_el[3

In [3]:
datasets = {
    'ETTh1': './ETTDataset/' ,
    'DEWINDh_small': './WINDataset/' ,
    'SYNTHh1': './SYNTHDataset/' ,
    'SYNTH_additive' : './SYNTHDataset/' ,
    'SYNTH_additive_reversal' : './SYNTHDataset/' ,
    'SYNTH_multiplicative' : './SYNTHDataset/' ,
    'SYNTH_multiplicative_reversal' : './SYNTHDataset/'
}
pred_lens = [24, 48, 168, 336, 720]

models = [CrossformerTS]

In [4]:
%%time 

for transformermodel in models:
    print(f'Model in use: {transformermodel.__name__}')
    model = transformermodel()
    model.compile(learning_rate=1e-4, loss='mse', early_stopping_patience=3)

    for iter in range(model.args.itr):
        for dataset_name, dataset_path in datasets.items():
            for pred_len in pred_lens:
                print(f"Training {transformermodel.__name__} on {dataset_name} with pred_len {pred_len}")

                # High amount of epochs to accomodate all models
                # Early stopping should kick in anyways
                model.fit(
                    iter = iter +1,
                    data=dataset_name,
                    data_root_path=dataset_path,
                    batch_size=32,
                    epochs=20, 
                    pred_len=pred_len
                )    

                predictions = model.predict()

                print(f'\nMoving to next...\n')




Model in use: CrossformerTS
Training CrossformerTS on ETTh1 with pred_len 24
Beginning to fit the model with the following arguments:
{'checkpoints': './checkpoints/', 'seq_len': 168, 'seg_len': 6, 'win_size': 2, 'factor': 10, 'data_dim': 7, 'd_model': 256, 'd_ff': 512, 'n_heads': 4, 'e_layers': 3, 'dropout': 0.2, 'baseline': False, 'num_workers': 0, 'lradj': 'type1', 'itr': 3, 'save_pred': True, 'use_gpu': True, 'use_multi_gpu': False, 'gpu': 0, 'devices': '0,1,2,3', 'inverse': False, 'learning_rate': 0.0001, 'loss': 'mse', 'patience': 3, 'data': 'ETTh1', 'root_path': './ETTDataset/', 'data_path': 'ETTh1.csv', 'train_epochs': 20, 'batch_size': 32, 'pred_len': 24, 'iter': 2, 'data_split': [8640, 2880, 2880]}
Use GPU: cuda:0
>>>>>>>start training : Crossformer_ETTh1_il168_pl24_sl6_win2_fa10_dm256_nh4_el3_iter2>>>>>>>>>>>>>>>>>>>>>>>>>>
train 8449
val 2857
test 2857
	iters: 100, epoch: 1 | loss: 0.5214266
	speed: 0.0956s/iter; left time: 495.3283s
	iters: 200, epoch: 1 | loss: 0.3553054


In [ ]:
datasets = {
    'ETTh1': ['./ETTDataset/' , 'M', 'OT' ,96] ,
    'DEWINDh_small': ['./WINDataset/' , 'S' , 'TARGET' , 168] ,
    'SYNTHh1': ['./SYNTHDataset/', 'S' , 'TARGET' , 168] ,
    'SYNTH_additive' : ['./SYNTHDataset/', 'S', 'TARGET' , 168] ,
    'SYNTH_additive_reversal' : ['./SYNTHDataset/', 'S', 'TARGET' , 168] ,
    'SYNTH_multiplicative' : ['./SYNTHDataset/', 'S', 'TARGET' , 168] ,
    'SYNTH_multiplicative_reversal' : ['./SYNTHDataset/' , 'S', 'TARGET' , 168]
}
pred_lens = [24, 48, 168, 336, 720]

models = [TSMixer]

In [7]:
%%time

# Simple loop
for transformermodel in models:
    print(f'Model in use: {transformermodel.__name__}')
    model = transformermodel()
    model.compile(learning_rate=1e-4, loss='mse', early_stopping_patience=3)
    for iter in range(model.args.itr):
        for dataset_name, dataset_params in datasets.items():
            for pred_len in pred_lens:
                print(f"Training {transformermodel.__name__} on {dataset_name} with pred_len {pred_len}")

                    # High amount of epochs to accomodate all models
                    # Early stopping should kick in anyways
                model.fit(
                    iter = iter + 1,
                    data=dataset_name,
                    data_root_path=dataset_params[0],
                    batch_size=32,
                    epochs=20, # very high
                    pred_len=pred_len,
                    features = dataset_params[1],
                    target = dataset_params[2],
                    seq_len = dataset_params[3],
                )

                predictions = model.predict()

                print(f'\nMoving to next...\n')


Model in use: LogsparseTS
Training LogsparseTS on ETTh1 with pred_len 24
Beginning to fit the model with the following arguments:
{'model': 'Logsparse', 'plot_flat': 0, 'verbose': 1, 'is_training': 1, 'inverse': False, 'data': 'ETTh1', 'root_path': './ETTDataset/', 'data_path': 'ETTh1.csv', 'target': 'OT', 'freq': 'h', 'checkpoints': './checkpoints/', 'checkpoint_flag': 1, 'features': 'M', 'seq_len': 768, 'label_len': 48, 'pred_len': 24, 'enc_in': 1, 'dec_in': 1, 'c_out': 1, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'factor': 3, 'distil': True, 'dropout': 0.05, 'embed': 'timeF', 'activation': 'gelu', 'output_attention': False, 'win_len': 6, 'res_len': None, 'qk_ker': 4, 'v_conv': 0, 'sparse_flag': 1, 'top_keys': 0, 'kernel_size': 3, 'train_strat_lstm': 'recursive', 'model_id': 'Logsparse_Synth1_24', 'num_workers': 0, 'itr': 3, 'train_epochs': 20, 'batch_size': 32, 'patience': 3, 'learning_rate': 0.0001, 'lr_decay_rate': 0.8, 'des': 'test', 'loss': 'mse',

RuntimeError: Error(s) in loading state_dict for Logsparse:
	size mismatch for enc_embedding.value_embedding.tokenConv.weight: copying a param with shape torch.Size([512, 7, 3]) from checkpoint, the shape in current model is torch.Size([512, 1, 3]).
	size mismatch for dec_embedding.value_embedding.tokenConv.weight: copying a param with shape torch.Size([512, 7, 3]) from checkpoint, the shape in current model is torch.Size([512, 1, 3]).
	size mismatch for decoder.projection.weight: copying a param with shape torch.Size([7, 512]) from checkpoint, the shape in current model is torch.Size([1, 512]).
	size mismatch for decoder.projection.bias: copying a param with shape torch.Size([7]) from checkpoint, the shape in current model is torch.Size([1]).

In [ ]:
datasets = {
    'SYNTHh1': './SYNTHDataset/',
    'DEWINDh_small': './WINDataset/' ,
    'SYNTH_additive' : './SYNTHDataset/' ,
    'SYNTH_additive_reversal' : './SYNTHDataset/' ,
    'SYNTH_multiplicative' : './SYNTHDataset/',
    'SYNTH_multiplicative_reversal' : './SYNTHDataset/' 
}
pred_lens = [24, 48, 168, 336, 720]

models = [PyraformerTS]

#result_dict_pyraformer = {}

for transformermodel in models:
    print(f'Model in use: {transformermodel.__name__}')
    model = transformermodel()
    model.compile(learning_rate=1e-4, loss='mse', early_stopping_patience=3)

    for dataset_name, dataset_path in datasets.items():
        for pred_len in pred_lens:
            print(f"Training {transformermodel.__name__} on {dataset_name} with pred_len {pred_len}")

            # High amount of epochs to accomodate all models
            # Early stopping should kick in anyways
            model.fit(
                data=dataset_name,
                data_root_path=dataset_path,
                batch_size=32,
                epochs=8, 
                pred_len=pred_len
            )

            #pred_name = 'prediction_' + f'{transformermodel.__name__}_{dataset_name}_{pred_len}'
            #true_name = 'true_' + f'{transformermodel.__name__}_{dataset_name}_{pred_len}'
            

            predictions , trues = model.predict()

            #result_dict_pyraformer[pred_name] = predictions
            #result_dict_pyraformer[true_name] = trues
            

            print(f'\nMoving to next...\n')


